# Nationality Detection Model

**Import the Libraries**

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import time
import warnings
warnings.filterwarnings("ignore")

import cv2
from PIL import Image

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
print("DONE")


2025-07-15 10:09:34.810430: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752574174.833097     133 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752574174.840192     133 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


DONE


**Find out the path and read the CSV file**

In [12]:
# list all .csv files and show the full path
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename.endswith('.csv'):
            print(os.path.join(dirname, filename))


/kaggle/input/fairface/fairface/fairface_label_val.csv
/kaggle/input/fairface/fairface/fairface_label_train.csv


In [18]:
train_csv = '/kaggle/input/fairface/fairface/fairface_label_val.csv'
val_csv = '/kaggle/input/fairface/fairface/fairface_label_train.csv'

df_train = pd.read_csv(train_csv)
df_val = pd.read_csv(val_csv)

# Preview
print(df_train.head())


        file    age  gender             race  service_test
0  val/1.jpg    3-9    Male       East Asian         False
1  val/2.jpg  50-59  Female       East Asian          True
2  val/3.jpg  30-39    Male            White          True
3  val/4.jpg  20-29  Female  Latino_Hispanic          True
4  val/5.jpg  20-29    Male  Southeast Asian         False


**Ensures the Dataset Healt and Summary**

In [11]:
# Checkinf that how many samples exist for every race label (our target)
print("Race Label Distribution in Training Data:")
print(df_train['race'].value_counts())

# Checking that if any missing values across the columns
print("\n Missing Values in Training Data:")
print(df_train.isnull().sum())

# Also, check that the how many number of samples are present in dataset
print("\n Total Training Samples:", len(df_train))


Race Label Distribution in Training Data:
race
White              2085
Latino_Hispanic    1623
Black              1556
East Asian         1550
Indian             1516
Southeast Asian    1415
Middle Eastern     1209
Name: count, dtype: int64

 Missing Values in Training Data:
file            0
age             0
gender          0
race            0
service_test    0
dtype: int64

 Total Training Samples: 10954


In [19]:
# Remove service test rows which true
df_train = df_train[df_train['service_test'] == False].reset_index(drop=True)
df_val = df_val[df_val['service_test'] == False].reset_index(drop=True)



**Converting the Race Column value in Numeric form for model understanding**

In [20]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df_train['race_label'] = label_encoder.fit_transform(df_train['race'])
df_val['race_label'] = label_encoder.transform(df_val['race'])

# Show the encoded class mapping
race_classes = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Encoded Race Classes:")
print(race_classes)


Encoded Race Classes:
{'Black': 0, 'East Asian': 1, 'Indian': 2, 'Latino_Hispanic': 3, 'Middle Eastern': 4, 'Southeast Asian': 5, 'White': 6}


Load and Pair Images with Labels

In [22]:
# define the image directory
train_img_dir = "/kaggle/input/fairface/train"
val_img_dir = "/kaggle/input/fairface/val"

# Add exact or full image paths to the DataFrames for new img_path(column)
df_train['img_path'] = df_train['file'].apply(lambda x: os.path.join(train_img_dir, x))
df_val['img_path'] = df_val['file'].apply(lambda x: os.path.join(val_img_dir, x))

# Check the changes
df_train.head()


,file,age,gender,race,service_test,race_label,img_path
0,val/1.jpg,3-9,Male,East Asian,False,1,/kaggle/input/fairface/train/val/1.jpg
1,val/5.jpg,20-29,Male,Southeast Asian,False,5,/kaggle/input/fairface/train/val/5.jpg
2,val/6.jpg,30-39,Male,Latino_Hispanic,False,3,/kaggle/input/fairface/train/val/6.jpg
3,val/8.jpg,3-9,Male,East Asian,False,1,/kaggle/input/fairface/train/val/8.jpg
4,val/9.jpg,20-29,Male,Southeast Asian,False,5,/kaggle/input/fairface/train/val/9.jpg
